<a href="https://colab.research.google.com/github/gabtns/Proyectos-de-Machine-Learning/blob/main/Detectores_de_manos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Se busca realizar la detecciones de manos. Para ello se invoca a la libreria cv2 y mediapipe.
Para realizar esto hace una clase y las funciones correspondientes para realizar la deteccion.


***Realizado por Gabriel Alegre, 01/07/2023***

In [ ]:
import cv2
import mediapipe as mp
from collections import namedtuple

class handDetector:
    def __init__(self,mode=False,max_num_hands=2,min_detection=0.2,min_tracking=0.2):
      '''Al buscar mpHands.Hands() podemos ver una serie de parametros los cuales
      son los mismo que se usan en esta clase, solo que se ajustan a lo requerido.  '''
        self.mode = mode
        self.max_num_hands = max_num_hands
        self.min_detection = min_detection
        self.min_tracking = min_tracking
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands()
        self.mkDraw = mp.solutions.drawing_utils


    def findHands(self,img,draw=True):
        ''' En este metodo se obtiene un parametro que en este caso se toma la camara, el cual se hace la conversion y se procesa la misma
        con el objeto realizado, iteramos sobre el mismo para obtener los valores que liuego podemos procesar y visualizar


        El metodo draw_landmarks lo que hace es la union entre los puntos sobre la imagen '''
        imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        self.result = self.hands.process(imgRGB)

        if self.result.multi_hand_landmarks:
            for detection in self.result.multi_hand_landmarks:
                if draw:
                    self.mkDraw.draw_landmarks(img,detection,self.mpHands.HAND_CONNECTIONS)

        return img

    def findPosition(self,img, handNo = 0,draw = True):
      ''' Se crea una lista la cual contendra los valores que se obtengan de la deteccion.

      Se valida que haya puntos, y se guarda los valores para la detectar la cantidad de manos

      Se toma los valores de la imagen, y luego se itera para obtener el ID y los ejes x,y,z y la visibilidad, se agrega a la lista y retorna la misma '''
        limslist= []
        if self.result.multi_hand_landmarks:
            myhand= self.result.multi_hand_landmarks[handNo]
            for id,num in enumerate(myhand.landmark):

                h,w,c = img.shape
                cx,cy = int(num.x * w),int(num.y * h)
                limslist.append([id,cx,cy])
                if draw:
                    cv2.circle(img,(cx,cy),7,(255,0,255),cv2.FILLED)
        return limslist


def main():
  '''  Por ultimo se obtiene el objeto donde esta la imagen(que en este caso es una prueba de un video), se crea el objeto donde estara la clase handDetector()


  Luego se realiza un bucle para leer la imagen y poder encontrar los valores de las manos que percibe el codigo, para salir de la imagen simplemente se aprieta la tecla "q" y sale del bucle.  '''
    ptime = 0
    ctime = 0
    cap = cv2.VideoCapture("proyecto_seguimiento\walking-m-p4.mp4")
    detector = handDetector()
    while True:
        res,img = cap.read()
        img = detector.findHands(img)
        limslist = detector.findPosition(img)
        ''' En esta lista se visualiza el punto numero 4, el cual solo se obtendran los valores de ese punto omitiendo los demas.
        La lista retornada tiene los valores de todos los puntos de la mano que son del 1 al 20, el 4 por ejemplo es el PULGAR'''
        if len(limslist) != 0:
            print(limslist[4])

        cv2.imshow("frame",img)
        if cv2.waitKey(1) == ord('q'):
            break

if __name__ == "__main__":
    main()